## Construcción y Entrenamiento del Modelo

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Función para cargar y procesar imágenes de entrenamiento
def load_and_preprocess_images(data, img_size=(48, 48), path_col='path', label_col='label', filter_labels=None):
    images = []
    labels = []
    for index, row in data.iterrows():
        if filter_labels is None or row[label_col] in filter_labels:
            img_path = row[path_col].replace('..', 'data').replace('\\', '/')
            img = load_img(img_path, target_size=img_size, color_mode='grayscale')
            img_array = img_to_array(img)
            images.append(img_array)
            labels.append(row[label_col])
    images = np.array(images, dtype='float32') / 255.0
    return images, labels

In [3]:
# Función para cargar y procesar imágenes sin etiquetas (para el conjunto de prueba)
def load_and_preprocess_images2(data, img_size=(48, 48)):
    images = []
    for index, row in data.iterrows():
        img_path = f"data/data/images/test/{row['id_img']}.jpg"
        img = load_img(img_path, target_size=img_size, color_mode='grayscale')
        img_array = img_to_array(img)
        images.append(img_array)
    images = np.array(images, dtype='float32') / 255.0
    return images

In [4]:
# Cargar los datos
train_data = pd.read_csv('data/train_set.csv')
test_data = pd.read_csv('test_set.csv')
sample_submission = pd.read_csv('sample_submision.csv')

In [5]:
# Ajustar los nombres de las columnas según sea necesario
path_col = 'path'  # Cambia esto si el nombre de la columna es diferente
label_col = 'label'

In [6]:
# Mostrar las columnas de los DataFrames
print("Columnas de train_set.csv:", train_data.columns)
print("Columnas de test_set.csv:", test_data.columns)

Columnas de train_set.csv: Index(['id_img', 'path', 'label'], dtype='object')
Columnas de test_set.csv: Index(['id_img'], dtype='object')


In [7]:
# Cargar y procesar imágenes de entrenamiento
train_images, train_labels = load_and_preprocess_images(train_data, path_col=path_col, label_col=label_col, filter_labels=['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise'])

In [8]:
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
train_labels_categorical = to_categorical(train_labels_encoded, num_classes=7)

In [9]:
# Dividir los datos en conjunto de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels_categorical, test_size=0.2, random_state=42)

In [10]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
datagen.fit(X_train)

In [11]:
# Definir la red neuronal
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [ ]:
# Compilar el modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Entrenar el modelo
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

In [13]:
history = model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=30, validation_data=(X_val, y_val))

Epoch 1/30
  1/303 ━━━━━━━━━━━━━━━━━━━━ 7:34 2s/step - accuracy: 0.8438 - loss: 0.4460

C:\Users\buque\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


303/303 ━━━━━━━━━━━━━━━━━━━━ 17s 50ms/step - accuracy: 0.8168 - loss: 0.3977 - val_accuracy: 0.8620 - val_loss: 0.3155
Epoch 2/30
303/303 ━━━━━━━━━━━━━━━━━━━━ 13s 43ms/step - accuracy: 0.8310 - loss: 0.3638 - val_accuracy: 0.8563 - val_loss: 0.3275
Epoch 3/30
303/303 ━━━━━━━━━━━━━━━━━━━━ 13s 42ms/step - accuracy: 0.8341 - loss: 0.3652 - val_accuracy: 0.8686 - val_loss: 0.3080
Epoch 4/30
303/303 ━━━━━━━━━━━━━━━━━━━━ 13s 43ms/step - accuracy: 0.8431 - loss: 0.3512 - val_accuracy: 0.8604 - val_loss: 0.3109
Epoch 5/30
303/303 ━━━━━━━━━━━━━━━━━━━━ 13s 44ms/step - accuracy: 0.8451 - loss: 0.3334 - val_accuracy: 0.8658 - val_loss: 0.3236
Epoch 6/30
303/303 ━━━━━━━━━━━━━━━━━━━━ 14s 45ms/step - accuracy: 0.8553 - loss: 0.3184 - val_accuracy: 0.8645 - val_loss: 0.3051
Epoch 7/30
303/303 ━━━━━━━━━━━━━━━━━━━━ 13s 41ms/step - accuracy: 0.8621 - loss: 0.3213 - val_accuracy: 0.8666 - val_loss: 0.3155
Epoch 8/30
303/303 ━━━━━━━━━━━━━━━━━━━━ 13s 42ms/step - accuracy: 0.8592 - loss: 0.3265 - val_accurac

In [14]:
# Evaluar el modelo
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation accuracy: {val_accuracy:.4f}')

76/76 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.8961 - loss: 0.2568
Validation accuracy: 0.8905


In [12]:
# Cargar y procesar imágenes de prueba
test_images = load_and_preprocess_images2(test_data)

In [ ]:
# Asegurarse de que las imágenes de prueba tengan la forma correcta
if test_images.ndim == 3:
    test_images = np.expand_dims(test_images, axis=-1)

In [ ]:
# Verificar la forma de las imágenes de prueba
print(f"Forma de las imágenes de prueba: {test_images.shape}")

In [ ]:
# Asegurarse de que no hay imágenes con dimensiones incorrectas
for i, img in enumerate(test_images):
    if img.shape != (48, 48, 1):
        print(f"Imagen en índice {i} tiene una forma incorrecta: {img.shape}")

In [26]:
# Realizar predicciones sobre el conjunto de prueba
predictions = model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)
predicted_labels_decoded = label_encoder.inverse_transform(predicted_labels)

221/221 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step


In [27]:
# Generar el archivo de submission
submission = sample_submission.copy()
submission['label'] = predicted_labels_decoded
submission.to_csv('submission_deeper_cnn,epochs20,batch_size32.csv', index=False)

In [28]:
submission

,id_img,label
0,10052,happy
1,10065,sad
2,10079,sad
3,10095,sad
4,10121,sad
...,...,...
7061,9806,sad
7062,9830,happy
7063,9853,sad
7064,9878,happy


In [13]:
def create_model(model_type='simple_cnn'):
    if model_type == 'simple_cnn':
        model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Conv2D(128, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(128, activation='relu'),
            Dropout(0.5),
            Dense(7, activation='softmax')
        ])
    elif model_type == 'deeper_cnn':
        model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
            Conv2D(32, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Conv2D(64, (3, 3), activation='relu'),
            Conv2D(64, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Conv2D(128, (3, 3), activation='relu'),
            Conv2D(128, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(256, activation='relu'),
            Dropout(0.5),
            Dense(7, activation='softmax')
        ])
    return model

In [35]:
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels_categorical, test_size=0.2, random_state=42)

In [14]:
# Configurar EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [15]:
# Parámetros para probar
model_types = ['simple_cnn', 'deeper_cnn']
epochs_list = [10, 20]
batch_sizes = [32, 64]

best_accuracy = 0
best_params = {}

# Entrenar y evaluar cada combinación de parámetros
for model_type in model_types:
    for epochs in epochs_list:
        for batch_size in batch_sizes:
            print(f"Entrenando modelo: {model_type}, epochs: {epochs}, batch_size: {batch_size}")
            model = create_model(model_type=model_type)
            model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
            model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=1, validation_data=(X_val, y_val), callbacks=[early_stopping])
            val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
            print(f"Validation accuracy: {val_accuracy:.4f}")
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                best_params = {'model_type': model_type, 'epochs': epochs, 'batch_size': batch_size}
                best_model = model

# Imprimir los mejores parámetros y el mejor score
print(f"Mejores Parámetros: {best_params}")
print(f"Mejor Score: {best_accuracy}")

Entrenando modelo: simple_cnn, epochs: 10, batch_size: 32
Epoch 1/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 31s 39ms/step - accuracy: 0.2505 - loss: 1.8207 - val_accuracy: 0.3759 - val_loss: 1.6038
Epoch 2/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 28s 39ms/step - accuracy: 0.3729 - loss: 1.5987 - val_accuracy: 0.4342 - val_loss: 1.4337
Epoch 3/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 28s 39ms/step - accuracy: 0.4316 - loss: 1.4604 - val_accuracy: 0.4767 - val_loss: 1.3836
Epoch 4/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 28s 39ms/step - accuracy: 0.4674 - loss: 1.3857 - val_accuracy: 0.4905 - val_loss: 1.3132
Epoch 5/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 28s 39ms/step - accuracy: 0.4889 - loss: 1.3272 - val_accuracy: 0.5176 - val_loss: 1.2759
Epoch 6/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 29s 40ms/step - accuracy: 0.5133 - loss: 1.2760 - val_accuracy: 0.5226 - val_loss: 1.2577
Epoch 7/10
721/721 ━━━━━━━━━━━━━━━━━━━━ 29s 40ms/step - accuracy: 0.5272 - loss: 1.2385 - val_accuracy: 0.5258 - val_loss: 1.2525
Epoch 8/10
721/721 ━━━━━━━━━━━━━

In [16]:
# Realizar predicciones sobre el conjunto de prueba
predictions = best_model.predict(test_images)
predicted_labels = np.argmax(predictions, axis=1)
predicted_labels_decoded = label_encoder.inverse_transform(predicted_labels)

221/221 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step


In [17]:
# Generar el archivo de submission con el nombre del modelo
submission_name = f"submission_{best_params['model_type']}.csv"
submission = sample_submission.copy()

In [18]:
# Asegurar que el submission contenga las columnas para las 7 emociones
submission['label'] = predicted_labels_decoded

submission.to_csv(f'data/submission/{submission_name}', index=False)

print(f"Submission file saved as {submission_name}")

Submission file saved as submission_simple_cnn.csv


In [19]:
print(submission)

      id_img     label
0      10052   neutral
1      10065       sad
2      10079     angry
3      10095     angry
4      10121     angry
...      ...       ...
7061    9806  surprise
7062    9830     happy
7063    9853  surprise
7064    9878     happy
7065     993  surprise

[7066 rows x 2 columns]


In [20]:
submission['label'].value_counts()

label
happy       1816
sad         1533
neutral     1492
surprise     962
angry        857
fear         402
disgust        4
Name: count, dtype: int64